# AutoGluon Tabular - Essential Functionality

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/master/docs/tutorials/tabular/tabular-essentials.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/master/docs/tutorials/tabular/tabular-essentials.ipynb)


Via a simple `fit()` call, AutoGluon can produce highly-accurate models to predict the values in one column of a data table based on the rest of the columns' values. Use AutoGluon with tabular data for both classification and regression problems. This tutorial demonstrates how to use AutoGluon to produce a classification model that predicts whether or not a customer of a bank exited the bank ("Exited" = 1). 

First, we need to install AutoGluon using the below code. This should take 1-2 minutes:

In [1]:
!pip install -q uv
# Use CPU version of PyTorch for faster installation
!uv pip install torch==2.5 torchvision --index-url https://download.pytorch.org/whl/cpu
!uv pip install autogluon==1.2

Using Python 3.10.15 environment at: /home/ec2-user/anaconda3/envs/python3
Resolved 12 packages in 132ms                                        
Prepared 3 packages in 2.21s                                             
Uninstalled 1 package in 171ms
Installed 3 packages in 115ms                               
 - sympy==1.13.3 (from file:///home/conda/feedstock_root/build_artifacts/sympy_1728484478345/work)
 + sympy==1.13.1
 + torch==2.5.0+cpu
 + torchvision==0.20.0+cpu
Using Python 3.10.15 environment at: /home/ec2-user/anaconda3/envs/python3
Resolved 206 packages in 1.65s                                       
Prepared 107 packages in 37.63s                                          
Uninstalled 6 packages in 231ms
Installed 107 packages in 1.36s                             
 + absl-py==2.1.0
 + accelerate==0.34.2
 + adagio==0.2.6
 + aiohttp-cors==0.7.0
 + antlr4-python3-runtime==4.9.3
 + appdirs==1.4.4
 + autogluon==1.2
 + autogluon-common==1.2
 + autogluon-core==1.2
 + autogluon-feat

## TabularPredictor

To start, import AutoGluon's [TabularPredictor](https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.html) and [TabularDataset](https://auto.gluon.ai/stable/api/autogluon.core.TabularDataset.html) classes:

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

Load training data from a [CSV file](https://en.wikipedia.org/wiki/Comma-separated_values) into an AutoGluon Dataset object. This object is a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) and the same methods can be applied to both.

In [3]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/bank_churn/train.csv')
train_data

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,54860,15701364,Hsueh,635,France,Male,38.0,5,0.00,1,1.0,1.0,166515.96,0
1,160522,15634974,Nwoye,697,France,Male,44.0,6,0.00,2,1.0,0.0,143671.54,0
2,56439,15706810,Smith,577,Germany,Male,39.0,3,153639.11,2,0.0,1.0,171519.06,0
3,144732,15783526,Okechukwu,756,France,Female,34.0,7,0.00,2,1.0,0.0,143393.24,0
4,134400,15641416,Chiabuotu,613,Germany,Female,32.0,3,96381.68,1,1.0,1.0,138871.12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,36050,15765322,Onyekachi,559,France,Male,28.0,0,0.00,2,1.0,1.0,82870.75,0
996,2789,15674929,K?,681,France,Female,29.0,7,87866.84,1,1.0,1.0,57858.25,0
997,55968,15589488,H?,713,France,Female,34.0,10,0.00,2,1.0,0.0,166720.28,0
998,99054,15676519,George,615,France,Male,61.0,9,0.00,2,0.0,0.0,150227.85,0


Note that we loaded data from a CSV file stored in the cloud. You can also specify a local file-path instead if you have already downloaded the CSV file to your own machine (e.g., using [wget](https://www.gnu.org/software/wget/)).
Each row in the table `train_data` corresponds to a single training example. In this particular dataset, each row corresponds to an individual person, and the columns contain various characteristics related to their bank account.

Let's first use these features to predict whether the person has left the bank, which is recorded in the `Exited` column of this table.

In [4]:
label = 'Exited'
print(f"Unique classes: {list(train_data[label].unique())}")

Unique classes: [0, 1]


AutoGluon works with raw data, meaning you don't need to perform any data preprocessing before fitting AutoGluon. We actively recommend that you avoid performing operations such as missing value imputation or one-hot-encoding, as AutoGluon has dedicated logic to handle these situations automatically. You can learn more about AutoGluon's preprocessing in the [Feature Engineering Tutorial](https://auto.gluon.ai/stable/tutorials/tabular/tabular-feature-engineering.html).

> **Note**: In this tutorial we will use a synthetic dataset for demonstration purposes. For this reason, we recommend focusing on the functionality and not the specific scores achieved by different models. For examples of AutoGluon's performance "in the wild", see [this page](https://github.com/autogluon/autogluon/blob/master/AWESOME.md#competition-solutions-using-autogluon).

### Training

Now we initialize and fit AutoGluon's TabularPredictor in one line of code:

In [5]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20241203_235857"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Oct 23 17:17:00 UTC 2024
CPU Count:          16
Memory Avail:       59.35 GB / 61.79 GB (96.0%)
Disk Space Avail:   74.55 GB / 78.56 GB (94.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.


That's it! We now have a TabularPredictor that is able to make predictions on new data.

### Prediction

Next, we can load a separate test data to demonstrate how to make predictions on new examples at inference time:

In [6]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/bank_churn/test.csv')
test_data

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/bank_churn/test.csv | Columns = 14 / 14 | Rows = 8000 -> 8000


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1934,15667938,Chikezie,539,France,Female,63.0,3,0.00,2,1.0,1.0,60552.44,0
1,35397,15661591,Dilke,652,Germany,Female,43.0,9,142706.10,1,0.0,0.0,108876.75,1
2,116092,15778196,Tsou,629,Germany,Female,33.0,4,165931.11,2,0.0,0.0,162719.39,0
3,102475,15732437,Briggs,708,Germany,Male,31.0,0,107884.81,1,0.0,1.0,183487.98,0
4,65633,15660147,Manna,716,France,Male,48.0,0,0.00,2,0.0,1.0,97640.20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,21373,15689341,Yermakov,666,France,Female,37.0,5,0.00,2,0.0,1.0,55470.54,0
7996,120270,15612776,Ritchie,627,Spain,Female,31.0,6,0.00,1,1.0,0.0,182494.78,0
7997,74802,15606839,Toscani,633,France,Female,42.0,5,0.00,1,1.0,1.0,96940.40,0
7998,94895,15676835,Yancy,637,Germany,Male,37.0,9,110581.29,1,1.0,1.0,133537.53,0


We can now use our trained models to make predictions on the new data:

In [7]:
y_pred = predictor.predict(test_data)
y_pred.head()  # Predictions

0    0
1    1
2    0
3    0
4    0
Name: Exited, dtype: int64

In [8]:
y_pred_proba = predictor.predict_proba(test_data)
y_pred_proba.head()  # Prediction Probabilities

,0,1
0,0.721154,0.278846
1,0.241600,0.758400
2,0.822953,0.177047
3,0.923047,0.076953
4,0.925227,0.074773


### Evaluation

Next, we can [evaluate](https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.evaluate.html) the predictor on the (labeled) test data:

In [9]:
predictor.evaluate(test_data)

{'accuracy': 0.8525,
 'balanced_accuracy': 0.7223728105291086,
 'mcc': 0.5142830099496976,
 'roc_auc': 0.863386681333925,
 'f1': 0.5877009084556254,
 'precision': 0.7194183062446535,
 'recall': 0.496751329001772}

We can also [evaluate each model individually](https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.leaderboard.html):

In [10]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,0.852500,0.915,accuracy,0.012424,0.002254,1.973634,0.012424,0.002254,1.973634,1,True,3
1,WeightedEnsemble_L2,0.852500,0.915,accuracy,0.013561,0.002881,2.032998,0.001137,0.000626,0.059364,2,True,14
2,LightGBM,0.852375,0.900,accuracy,0.006659,0.001771,0.281493,0.006659,0.001771,0.281493,1,True,4
3,RandomForestGini,0.850875,0.900,accuracy,0.074139,0.036183,0.587501,0.074139,0.036183,0.587501,1,True,5
4,RandomForestEntr,0.849625,0.900,accuracy,0.064119,0.036482,0.446140,0.064119,0.036482,0.446140,1,True,6
5,CatBoost,0.849250,0.910,accuracy,0.006660,0.002107,0.627876,0.006660,0.002107,0.627876,1,True,7
6,XGBoost,0.847125,0.900,accuracy,0.016810,0.004282,0.214654,0.016810,0.004282,0.214654,1,True,11
7,ExtraTreesEntr,0.846250,0.885,accuracy,0.066594,0.036728,0.453235,0.066594,0.036728,0.453235,1,True,9
8,ExtraTreesGini,0.846125,0.890,accuracy,0.068374,0.036785,0.454847,0.068374,0.036785,0.454847,1,True,8
9,LightGBMLarge,0.838125,0.895,accuracy,0.011410,0.002365,0.622459,0.011410,0.002365,0.622459,1,True,13


### Loading a Trained Predictor

Finally, we can load the predictor in a new session (or new machine) by calling [TabularPredictor.load()](https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.load.html) and specifying the location of the predictor artifact on disk.

In [11]:
predictor.path  # The path on disk where the predictor is saved

'/home/ec2-user/SageMaker/autogluon-workshops/2024-reinvent/tabular/AutogluonModels/ag-20241203_235857'

In [12]:
# Load the predictor by specifying the path it is saved to on disk.
# You can control where it is saved to by setting the `path` parameter during init
predictor = TabularPredictor.load(predictor.path)

```{warning}

`TabularPredictor.load()` uses the `pickle` module implicitly, which is known to be insecure. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling. Never load data that could have come from an untrusted source, or that could have been tampered with. **Only load data you trust.**

```

Now you're ready to try AutoGluon on your own tabular datasets!
As long as they're stored in a popular format like CSV, you should be able to achieve strong predictive performance with just 2 lines of code:

```
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=<variable-name>).fit(train_data=<file-name>)
```


**Note:** This simple call to [TabularPredictor.fit()](https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.fit.html) is intended for your first prototype model. In a subsequent section, we'll demonstrate how to maximize predictive performance by additionally specifying the `presets` parameter to `fit()` and the `eval_metric` parameter to `TabularPredictor()`.

## Description of fit()

Here we discuss what happened during `fit()`.

Since there are only two possible values of the `Exited` variable, this was a binary classification problem, for which an appropriate performance metric is _accuracy_. AutoGluon automatically infers this as well as the type of each feature (i.e., which columns contain continuous numbers vs. discrete categories). AutoGluon can also automatically handle common issues like missing data and rescaling feature values.

We did not specify separate validation data and so AutoGluon automatically chose a random training/validation split of the data. The data used for validation is separated from the training data and is used to determine the models and hyperparameter-values that produce the best results. Rather than just a single model, AutoGluon trains multiple models and ensembles them together to obtain superior predictive performance.

By default, AutoGluon tries to fit [various types of models](https://auto.gluon.ai/stable/api/autogluon.tabular.models.html) including neural networks and tree ensembles. Each type of model has various hyperparameters, which traditionally, the user would have to specify. AutoGluon automates this process.

AutoGluon automatically and iteratively tests values for hyperparameters to produce the best performance on the validation data. This involves repeatedly training models under different hyperparameter settings and evaluating their performance. This process can be computationally-intensive, so `fit()` parallelizes this process across multiple threads using [Ray](https://www.ray.io/). To control runtimes, you can specify various arguments in `fit()` such as `time_limit` as demonstrated in the subsequent **[In-Depth Tutorial](https://auto.gluon.ai/stable/tutorials/tabular/tabular-indepth.html)**.

We can view what properties AutoGluon automatically inferred about our prediction task:

In [13]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  binary
AutoGluon identified the following types of features:
('category', [])  : 2 | ['Surname', 'Geography']
('float', [])     : 3 | ['Age', 'Balance', 'EstimatedSalary']
('int', [])       : 5 | ['id', 'CustomerId', 'CreditScore', 'Tenure', 'NumOfProducts']
('int', ['bool']) : 3 | ['Gender', 'HasCrCard', 'IsActiveMember']


AutoGluon correctly recognized our prediction problem to be a **binary classification** task and decided that variables such as `Age` should be represented as numeric, whereas variables such as `Geography` should be represented as categorical objects. The `feature_metadata` attribute allows you to see the inferred data type of each predictive variable after preprocessing (this is its _raw_ dtype; some features may also be associated with additional _special_ dtypes if produced via feature-engineering, e.g. numerical representations of a datetime/text column).

To transform the data into AutoGluon's internal representation, we can do the following:

In [14]:
test_data_transform = predictor.transform_features(test_data)
test_data_transform.head()

,id,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Surname,Geography
0,1934,15667938,539,0,63.0,3,0.00,2,1,1,60552.44,NaN,0
1,35397,15661591,652,0,43.0,9,142706.10,1,0,0,108876.75,NaN,1
2,116092,15778196,629,0,33.0,4,165931.11,2,0,0,162719.39,173,1
3,102475,15732437,708,1,31.0,0,107884.81,1,0,1,183487.98,NaN,1
4,65633,15660147,716,1,48.0,0,0.00,2,0,1,97640.20,94,0


Notice how the data is purely numeric after pre-processing (although categorical features will still be treated as categorical downstream).

To better understand our trained predictor, we can estimate the overall importance of each feature via [TabularPredictor.feature_importance()](https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.feature_importance.html):

In [15]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 13 features using 5000 rows with 5 shuffle sets...
	1.14s	= Expected runtime (0.23s per shuffle set)
	0.7s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Age,0.05544,0.004175,3.829192e-06,5,0.064036,0.046844
NumOfProducts,0.05164,0.001936,2.366040e-07,5,0.055626,0.047654
IsActiveMember,0.02084,0.002747,3.542068e-05,5,0.026497,0.015183
Geography,0.00472,0.002373,5.635130e-03,5,0.009606,-0.000166
Gender,0.00456,0.002027,3.664753e-03,5,0.008733,0.000387
Balance,0.00160,0.002102,8.200984e-02,5,0.005929,-0.002729
id,0.00112,0.001591,9.531425e-02,5,0.004396,-0.002156
HasCrCard,0.00100,0.001192,6.691333e-02,5,0.003454,-0.001454
Surname,0.00000,0.000000,5.000000e-01,5,0.000000,0.000000
CustomerId,-0.00064,0.000555,9.692920e-01,5,0.000503,-0.001783


The `importance` column is an estimate for the amount the evaluation metric score would drop if the feature were removed from the data.
Negative values of `importance` mean that it is likely to improve the results if re-fit with the feature removed.

When we call `predict()`, AutoGluon automatically predicts with the model that displayed the best performance on validation data (i.e. the weighted-ensemble).

In [16]:
predictor.model_best

'WeightedEnsemble_L2'

We can instead specify which model to use for predictions like this:

```
predictor.predict(test_data, model='LightGBM')
```

You can get the list of trained models via `.leaderboard()` or `.model_names()`:

In [17]:
predictor.model_names()

['KNeighborsUnif',
 'KNeighborsDist',
 'LightGBMXT',
 'LightGBM',
 'RandomForestGini',
 'RandomForestEntr',
 'CatBoost',
 'ExtraTreesGini',
 'ExtraTreesEntr',
 'NeuralNetFastAI',
 'XGBoost',
 'NeuralNetTorch',
 'LightGBMLarge',
 'WeightedEnsemble_L2']

The scores of predictive performance above were based on a default evaluation metric (accuracy for binary classification). Performance in certain applications may be measured by different metrics than the ones AutoGluon optimizes for by default. If you know the metric that counts in your application, you should specify it via the `eval_metric` argument as demonstrated in the next section.

## Presets

AutoGluon comes with a variety of presets that can be specified in the call to `.fit` via the `presets` argument. `medium_quality` is used by default to encourage initial prototyping, but for serious usage, the other presets should be used instead.

| Preset         | Model Quality                                          | Use Cases                                                                                                                                               | Fit Time (Ideal) | Inference Time (Relative to medium_quality) | Disk Usage |
| :------------- |:-------------------------------------------------------| :------------------------------------------------------------------------------------------------------------------------------------------------------ |:-----------------| :------------------------------------------ | :--------- |
| best_quality   | State-of-the-art (SOTA), much better than high_quality | When accuracy is what matters                                                                                                                           | 16x+             | 32x+                                        | 16x+       |
| high_quality   | Better than good_quality                               | When a very powerful, portable solution with fast inference is required: Large-scale batch inference                                                    | 16x+             | 4x                                          | 2x         |
| good_quality   | Stronger than any other AutoML Framework               | When a powerful, highly portable solution with very fast inference is required: Billion-scale batch inference, sub-100ms online-inference, edge-devices | 16x              | 2x                                          | 0.1x       |
| medium_quality | Competitive with other top AutoML Frameworks           | Initial prototyping, establishing a performance baseline                                                                                                | 1x               | 1x                                          | 1x         |

We recommend users to start with `medium_quality` to get a sense of the problem and identify any data related issues. If `medium_quality` is taking too long to train, consider subsampling the training data during this prototyping phase.  
Once you are comfortable, next try `best_quality`. Make sure to specify at least 16x the `time_limit` value as used in `medium_quality`. Once finished, you should have a very powerful solution that is often stronger than `medium_quality`.  
Make sure to consider holding out test data that AutoGluon never sees during training to ensure that the models are performing as expected in terms of performance.  
Once you evaluate both `best_quality` and `medium_quality`, check if either satisfies your needs. If neither do, consider trying `high_quality` and/or `good_quality`.  
If none of the presets satisfy requirements, refer to [Predicting Columns in a Table - In Depth](https://auto.gluon.ai/stable/tutorials/tabular/tabular-indepth.html) for more advanced AutoGluon options.

## Maximizing predictive performance

**Note:** You should not call `fit()` with entirely default arguments if you are benchmarking AutoGluon-Tabular or hoping to maximize its accuracy!
To get the best predictive accuracy with AutoGluon, you should generally use it like this:

In [18]:
time_limit = 120  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20241204_000252"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Oct 23 17:17:00 UTC 2024
CPU Count:          16
Memory Avail:       58.89 GB / 61.79 GB (95.3%)
Disk Space Avail:   74.52 GB / 78.56 GB (94.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of

In [19]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.882181,0.888265,roc_auc,0.042804,0.023806,2.187929,0.042804,0.023806,2.187929,1,True,7
1,CatBoost_r177_BAG_L1,0.880098,0.880501,roc_auc,0.022286,0.023890,1.647939,0.022286,0.023890,1.647939,1,True,14
2,WeightedEnsemble_L2,0.878246,0.898842,roc_auc,1.415036,0.352609,21.064224,0.002825,0.000553,0.083542,2,True,19
3,WeightedEnsemble_L3,0.877265,0.900093,roc_auc,3.587722,0.978499,44.321019,0.003213,0.000591,0.092001,3,True,31
4,CatBoost_BAG_L2,0.876691,0.886882,roc_auc,2.994690,0.798231,39.017195,0.030808,0.032226,3.019979,2,True,24
5,CatBoost_r9_BAG_L1,0.876415,0.887894,roc_auc,0.057712,0.024676,7.125108,0.057712,0.024676,7.125108,1,True,18
6,ExtraTreesGini_BAG_L2,0.875745,0.879145,roc_auc,3.033794,0.835689,36.455195,0.069912,0.069684,0.457979,2,True,25
7,ExtraTreesEntr_BAG_L2,0.875503,0.880465,roc_auc,3.030683,0.835886,36.450021,0.066802,0.069880,0.452805,2,True,26
8,LightGBMXT_BAG_L2,0.875053,0.882441,roc_auc,3.015293,0.793603,37.027897,0.051412,0.027598,1.030681,2,True,20
9,NeuralNetTorch_BAG_L2,0.874786,0.875293,roc_auc,3.363585,0.895944,42.191571,0.399704,0.129939,6.194355,2,True,29


This command implements the following strategy to maximize accuracy:

- Specify the argument `presets='best_quality'`, which allows AutoGluon to automatically construct powerful model ensembles based on [stacking/bagging](https://arxiv.org/abs/2003.06505), and will greatly improve the resulting predictions if granted sufficient training time. The default value of `presets` is `'medium_quality'`, which produces _less_ accurate models but facilitates faster prototyping. With `presets`, you can flexibly prioritize predictive accuracy vs. training/inference speed. For example, if you care less about predictive performance and want to quickly deploy a basic model, consider using: `presets=['good_quality', 'optimize_for_deployment']`.

- Provide the parameter `eval_metric` to `TabularPredictor()` if you know what metric will be used to evaluate predictions in your application. Some other non-default metrics you might use include things like: `'f1'` (for binary classification), `'roc_auc'` (for binary classification), `'log_loss'` (for classification), `'mean_absolute_error'` (for regression), `'median_absolute_error'` (for regression). You can also define your own custom metric function. For more information refer to [Adding a custom metric to AutoGluon](https://auto.gluon.ai/stable/tutorials/tabular/advanced/tabular-custom-metric.html).

- Include all your data in `train_data` and do not provide `tuning_data` (AutoGluon will split the data more intelligently to fit its needs).

- Do not specify the `hyperparameter_tune_kwargs` argument (counterintuitively, hyperparameter tuning is not the best way to spend a limited training time budgets, as model ensembling is often superior). We recommend you only use `hyperparameter_tune_kwargs` if your goal is to deploy a single model rather than an ensemble.

- Do not specify the `hyperparameters` argument (allow AutoGluon to adaptively select which models/hyperparameters to use).

- Set `time_limit` to the longest amount of time (in seconds) that you are willing to wait. AutoGluon's predictive performance improves the longer `fit()` is allowed to run.

## Regression (predicting numeric table columns):

To demonstrate that `fit()` can also automatically handle regression tasks, we now try to predict the numeric `EstimatedSalary` variable in the same table based on the other features:

In [20]:
salary_column = 'EstimatedSalary'
train_data[salary_column].head()

0    166515.96
1    143671.54
2    171519.06
3    143393.24
4    138871.12
Name: EstimatedSalary, dtype: float64

We again call `fit()`, imposing a time-limit this time (in seconds), and also demonstrate a shorthand method to evaluate the resulting model on the test data (which contain labels):

In [21]:
predictor_salary = TabularPredictor(label=salary_column, path="agModels-predict-salary").fit(train_data, time_limit=60)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Oct 23 17:17:00 UTC 2024
CPU Count:          16
Memory Avail:       58.33 GB / 61.79 GB (94.4%)
Disk Space Avail:   74.40 GB / 78.56 GB (94.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with fast inference speed.
	presets='go

In [22]:
predictor_salary.evaluate(test_data)

{'root_mean_squared_error': -50344.53877121951,
 'mean_squared_error': -2534572584.0868244,
 'mean_absolute_error': -42728.012336640626,
 'r2': -0.0063539398953478265,
 'pearsonr': 0.0082774732713095,
 'median_absolute_error': -40101.941875000004}

Note that we didn't need to tell AutoGluon this is a regression problem, it automatically inferred this from the data and reported the appropriate performance metric (root-mean-squared-error (RMSE) by default). To specify a particular evaluation metric other than the default, set the `eval_metric` parameter of [TabularPredictor()](https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.html) and AutoGluon will tailor its models to optimize your metric (e.g. `eval_metric = 'mean_absolute_error'`). For evaluation metrics where higher values are worse (like RMSE), AutoGluon will flip their sign and print them as negative values during training (as it internally assumes higher values are better). You can even specify a custom metric by following the [Custom Metric Tutorial](https://auto.gluon.ai/stable/tutorials/tabular/advanced/tabular-custom-metric.html).

We can call leaderboard to see the per-model performance:

In [23]:
predictor_salary.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,-50191.544185,-49305.955285,root_mean_squared_error,0.002219,0.002328,0.376030,0.002219,0.002328,0.376030,1,True,4
1,LightGBMXT,-50212.123525,-49273.423889,root_mean_squared_error,0.002706,0.002227,0.361889,0.002706,0.002227,0.361889,1,True,3
2,LightGBMLarge,-50218.152211,-49131.174213,root_mean_squared_error,0.002746,0.002441,0.719947,0.002746,0.002441,0.719947,1,True,11
3,NeuralNetFastAI,-50265.412638,-49334.881521,root_mean_squared_error,0.087555,0.007036,0.615979,0.087555,0.007036,0.615979,1,True,8
4,XGBoost,-50327.849633,-49142.724415,root_mean_squared_error,0.014318,0.004384,0.282491,0.014318,0.004384,0.282491,1,True,9
5,WeightedEnsemble_L2,-50344.538771,-48926.670289,root_mean_squared_error,0.020038,0.007288,0.691897,0.001226,0.000480,0.012407,2,True,12
6,CatBoost,-50363.708161,-48928.190329,root_mean_squared_error,0.004494,0.002424,0.396999,0.004494,0.002424,0.396999,1,True,6
7,NeuralNetTorch,-50383.239094,-49355.076851,root_mean_squared_error,0.021342,0.005959,1.138548,0.021342,0.005959,1.138548,1,True,10
8,ExtraTreesMSE,-51110.974921,-51198.869052,root_mean_squared_error,0.077434,0.035965,0.413350,0.077434,0.035965,0.413350,1,True,7
9,RandomForestMSE,-51425.239334,-51457.026995,root_mean_squared_error,0.086206,0.036698,0.445432,0.086206,0.036698,0.445432,1,True,5


**Data Formats:** AutoGluon can currently operate on data tables already loaded into Python as pandas DataFrames, or those stored in files of [CSV format](https://en.wikipedia.org/wiki/Comma-separated_values) or [Parquet format](https://databricks.com/glossary/what-is-parquet). If your data lives in multiple tables, you will first need to join them into a single table whose rows correspond to statistically independent observations (datapoints) and columns correspond to different features (aka. variables/covariates).

Refer to the [TabularPredictor documentation](https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.html) to see all of the available methods/options.

## Next Steps

Please continue to the next workshop section on **"Tabular regression & classification/In-depth functionality"**.